In [ ]:
import numpy as np
import pandas as pd
import utils
import scoring
import catboost
from sklearn.model_selection import StratifiedKFold, KFold

## part 1

In [ ]:
full_train = pd.read_csv("data/train_part_1_v2.csv", dtype=utils.DTYPES, index_col='id')

In [ ]:
closest_hits_features = pd.read_csv('more_features_fixed_part_1.csv',dtype=np.float32)

In [ ]:
train_concat = pd.concat(
    [full_train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

## part 2

In [ ]:
full_train2 = pd.read_csv("data/train_part_2_v2.csv", dtype=utils.DTYPES, index_col='id')

In [ ]:
closest_hits_features2 = pd.read_csv('more_features_fixed_part_2.csv',dtype=np.float32)

In [ ]:
#fix indexes
a = closest_hits_features2.index.tolist()
a = list(np.array(a) + 2722852)
closest_hits_features2.index = a

In [ ]:
train_concat2 = pd.concat(
    [full_train2.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features2], axis=1)

## concat train data

In [ ]:
train_concat = pd.concat([train_concat, train_concat2], axis=0)

In [ ]:
train_concat.shape

## additional features

In [ ]:
for i in range(4):
    train_concat[f'plastina_{i}'] = np.sqrt(train_concat[f'MatchedHit_X[{i}]'] ** 2 + train_concat[f'MatchedHit_Y[{i}]'] ** 2 + train_concat[f'MatchedHit_Z[{i}]']**2)

In [ ]:
for i in range(4):
    train_concat.loc[(train_concat[f'MatchedHit_X[{i}]']==-9999)|(train_concat[f'MatchedHit_Y[{i}]']==-9999)|(train_concat[f'MatchedHit_Z[{i}]']==-9999),f'plastina_{i}'] = -9999

In [ ]:
for i in range(8):
    train_concat[f'arr_{i}'] = train_concat[f'{i}'] / train_concat[f'{16+i}']**2

In [ ]:
train_concat['angle'] = np.arccos(train_concat['PT'] / train_concat['P'])

In [ ]:
for i in range(4):
    train_concat[f'size_clusters_{i}']=train_concat[f'ncl[{i}]']*train_concat[f'avg_cs[{i}]']

In [ ]:
for i in range(4):
    train_concat[f'v_{i}'] = train_concat[f'plastina_{i}'] / train_concat[f'MatchedHit_T[{i}]']

## pretrain work

In [ ]:
labels = pd.concat([full_train.label, full_train2.label], axis=0)

In [ ]:
weights = pd.concat([full_train.weight, full_train2.weight],axis=0)

In [ ]:
def wrapper(value):
    if value < 0:
        value = 0
    return value
abs_weights = weights.apply(wrapper)

## train catboost model mix

In [ ]:
folds = StratifiedKFold(n_splits=5,shuffle=True,random_state=12)

In [ ]:
model = catboost.CatBoostClassifier(loss_function='Logloss',
                                    l2_leaf_reg=20,
                                    border_count=200,
                                    max_depth=5,
                                    bootstrap_type='No', 
                                    random_strength=0, 
                                    iterations=3500, 
                                    learning_rate=0.05, 
                                    verbose=False,
                                    thread_count=4, 
                                    task_type='GPU', 
                                    random_state=12,
                                    rsm=1,
)

In [ ]:
scores = []
for index, (trn_idx, val_idx) in enumerate(folds.split(train_concat.values,labels)):
    model.fit(train_concat.loc[trn_idx], labels[trn_idx], sample_weight=abs_weights[trn_idx], plot=True)
    validation_predictions = model.predict_proba(train_concat.loc[val_idx].values)[:, 1]
    scores.append(scoring.rejection90(labels[val_idx].values, validation_predictions, sample_weight=weights[val_idx].values))
    model.save_model(f'fold_{index}.cbm')

In [ ]:
scores